In [49]:
import re
import os
import numpy as np
import pandas as pd

from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
sns.set_context("notebook", font_scale=1)
matplotlib.style.use('ggplot')
%matplotlib inline 

import warnings
warnings.filterwarnings('ignore')

In [24]:
hdAct1 = 'AAAAGACATCTTTTCCAAATCAGTCAGTTCCAGCACCGACGCCACCCGGATTCCCCCGTCATCGCATCGCTGTTCGCTATTCATCCCTGCGCTATTTTTCACTTTAGTTTCGCGTAATCCCTCAGAACAGTCGCAATGGAAGACGAAGTTGCCGCCTTGGTCGTGGACAATGGATCCGGTATGTGCAAGGCCGGATTTGCCGGAGATGACGCTCCCCGCGCCGTCTTCCCCTCCATCGTTGGCCGACCCCGTCATCAGGGTGTCATGGTCGGTATGGGTCAAAAGGACAGCTACGTCGGTGATGAGGCCCAGAGCAAGCGCGGTATCCTGACGCTCAAGTACCCCATCGAGCACGGCATCGTCACCAACTGGGATGACATGGAGAAGATCTGGCATCACACCTTCTACAACGAGCTCCGCGTGGCTCCCGAGGAACACCCCGTCCTCCTGACTGAGGCTCCCCTCAACCCCAAGGCCAACAGGGAAAAGATGACCCAGATCATGTTCGAGACATTCAACACCCCCGCCATGTACGTCGCGATTCAGGCTGTGCTCTCCCTGTACGCGTCCGGTCGTACCACCGGTATCGTGCTGGACTCTGGTGATGGTGTCTCCCACACTGTCCCCATCTACGAAGGTTATGCTCTGCCTCACGCCATCCTCCGTCTGGATTTGGCCGGTCGCGACTTGACTGACTACTTGATGAAGATCCTGACTGAGCGCGGTACAGCTTCGTCACAACGCT'
hdMag1 = 'CCCATTAGAGGGATTTCATCTATTTCAACTCTTGGGCATTTTCTCTGACGCTTAAATTTTTCCCAGACAATCTCGAAGGAAAACGAAAATGACGGAGGACCGTTTCTACGTGCGATACTACGTCGGCCATAAAGGCAAGTTCGGGCATGAATTTTTGGAATTCGAATTTCGCCCGGACGGTCGCCTGCGTTACGCCAACAATTCCAACTACAAGAACGACACGATGATCCGCAAGGAGGTGGTCGTCCATCCGGCTATCTTGGAAGAGGTGAAGCGCATCATTCAGGACAGTGAGATTCTCCGGGAGAACGACGCCAAGTGGCCGCAGCCGGACCGCGTGGGCCGGCAGGAGTTAGAGATTCTGCTGGACGACGAGCACATCTCCTTCAACACGGGAAAGATCGGCTCGCTGATGGACGTCAACAACAGCCCCGATCCAGAAGGTCTGCGTTGCTTCTACTACCTCGTGCAGGATCTCAAGTGTCTCGTCTTCTCCCTCATTGCGCTGCACTTTAAAATCAAGCCGATTTGAACGCTTGTAATCAACCAATCAAGCGCGAAGATTTGCACTAGCACCCTCGTCGAC'

In [25]:
%%time 
df =  pd.read_csv('pam_sites.csv')

CPU times: user 302 ms, sys: 48.7 ms, total: 351 ms
Wall time: 584 ms


In [26]:
df.head()

,Contig,Start,End,gRNA_Seq,PAM,Strand,G_start,Unique
0,nHd.2.3.scaf00001,2,21,GGTGACTGTATTCCGGGGGG,AGG,+,True,No
1,nHd.2.3.scaf00001,2875,2894,GAATGCATTTCGTTGAATGG,AGG,+,True,No
2,nHd.2.3.scaf00001,3222,3241,GAATGCATTTCGCTGAATGG,AGG,+,True,No
3,nHd.2.3.scaf00001,3291,3310,GCGGGCATCTCATTGTCGGG,AGG,+,True,Yes
4,nHd.2.3.scaf00001,3594,3613,GCATGCGCTGTTTCCAACGG,AGG,+,True,No


In [27]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Start,239950,38268.9,51313.5,1,6532,20655.5,49273.8,594043
End,239950,38268.9,51313.5,4,6530,20660.5,49270.8,594024
G_start,239950,1,0,True,1,1,1,True


In [28]:
uniq_df = df[df.Unique=='Yes']
uniq_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Start,155748,40498.1,53579,1,7306,22306.5,52519,593404
End,155748,40498.2,53579,4,7305,22304.5,52517.8,593385
G_start,155748,1,0,True,1,1,1,True


In [39]:
def pairDNA(x):
    if(x=='A'):
        return 'T'
    elif(x=='T'):
        return 'A'
    elif(x=='C'):
        return 'G'
    else:
        return 'C'

def dnaComp(s):
    # Gets complementary strand of DNA
    return ''.join(map(lambda b: pairDNA(b), s))

In [42]:
uniq_df['gRNA_Seq_Comp'] = map(lambda x: dnaComp(x), uniq_df['gRNA_Seq'])

In [56]:
scaf209_df = uniq_df[uniq_df.Contig=='nHd.2.3.scaf00209']
display(scaf209_df.describe().T)
scaf20_df = uniq_df[uniq_df.Contig=='nHd.2.3.scaf00020']
display(scaf209_df.describe().T)

,count,mean,std,min,25%,50%,75%,max
Start,160,48657.5,28897.3,1953,23643.8,47753.5,73145.5,99529
End,160,48655.4,28895.9,1934,23624.8,47753.5,73126.5,99510
G_start,160,1,0,True,1,1,1,True


,count,mean,std,min,25%,50%,75%,max
Start,160,48657.5,28897.3,1953,23643.8,47753.5,73145.5,99529
End,160,48655.4,28895.9,1934,23624.8,47753.5,73126.5,99510
G_start,160,1,0,True,1,1,1,True


In [58]:
display(scaf209_df.sample(5))
display(scaf20_df.sample(5))

,Contig,Start,End,gRNA_Seq,PAM,Strand,G_start,Unique,gRNA_Seq_Comp
60471,nHd.2.3.scaf00209,16017,16036,GGGCATCAACGGCAAAGTGG,TGG,+,True,Yes,CCCGTAGTTGCCGTTTCACC
60630,nHd.2.3.scaf00209,60397,60378,GAATTTGAGGCATGTTATGG,AGG,-,True,Yes,CTTAAACTCCGTACAATACC
60670,nHd.2.3.scaf00209,88847,88828,GGTGAAGAGGGTCACTCCGG,GGG,-,True,Yes,CCACTTCTCCCAGTGAGGCC
60469,nHd.2.3.scaf00209,12700,12719,GCGCCCGATACTCCCGTCGG,CGG,+,True,Yes,CGCGGGCTATGAGGGCAGCC
60627,nHd.2.3.scaf00209,54919,54900,GCAATCCACATATTGTTTGG,AGG,-,True,Yes,CGTTAGGTGTATAACAAACC


,Contig,Start,End,gRNA_Seq,PAM,Strand,G_start,Unique,gRNA_Seq_Comp
11240,nHd.2.3.scaf00020,138397,138416,GAAGGAGTGAAATTGGGGGG,AGG,+,True,Yes,CTTCCTCACTTTAACCCCCC
11482,nHd.2.3.scaf00020,73577,73558,GTATCCGGAGCCATCCGTGG,TGG,-,True,Yes,CATAGGCCTCGGTAGGCACC
11555,nHd.2.3.scaf00020,140389,140370,GGCCATTCTGTTGGCCTGGG,AGG,-,True,Yes,CCGGTAAGACAACCGGACCC
11239,nHd.2.3.scaf00020,138147,138166,GACGTGCACCAGAGTGCGGG,TGG,+,True,Yes,CTGCACGTGGTCTCACGCCC
11610,nHd.2.3.scaf00020,192658,192639,GCGTGGTGGCGCGCTGCTGG,TGG,-,True,Yes,CGCACCACCGCGCGACGACC


In [52]:
def matchGRNA(gRNA,gene):
    match = re.findall(gRNA,gene)
    if(len(match)>0):
        return True
    else:
        return False

## Matches

In [ ]:
scaf209_df['Match_gRNA'] = map(lambda x: matchGRNA(x,hdAct1),scaf209_df['gRNA_Seq'])
scaf20_df['Match_gRNA'] = map(lambda x: matchGRNA(x,hdMag1),scaf20_df['gRNA_Seq'])

print(len(scaf209_df[scaf209_df['Match_gRNA']==True]))
print(len(scaf20_df[scaf20_df['Match_gRNA']==True]))

In [65]:
scaf209_df['Match_gRNA'] = map(lambda x: matchGRNA(x,hdAct1),scaf209_df['gRNA_Seq_Comp'])
scaf20_df['Match_gRNA'] = map(lambda x: matchGRNA(x,hdMag1),scaf20_df['gRNA_Seq_Comp'])

print(len(scaf209_df[scaf209_df['Match_gRNA']==True]))
print(len(scaf20_df[scaf20_df['Match_gRNA']==True]))

0
0
